In [6]:
import tkinter as tk
from tkinter import messagebox

class VendingMachine:
    def __init__(self):
        self.state = 'IDLE'
        self.stock = {'A1': {'name': 'Soda', 'quantity': 2, 'price': 1.00},
                      'B2': {'name': 'Chips', 'quantity': 1, 'price': 0.75},
                      'C3': {'name': 'Candy', 'quantity': 0, 'price': 0.50}}
        self.selected_product_code = None
        self.inserted_amount = 0.0

    def insert_coin(self, amount):
        if self.state == 'IDLE':
            if amount > 0:
                self.state = 'INSERTING_COIN'
                self.inserted_amount += amount
                return f"${amount:.2f} inserted. Current amount: ${self.inserted_amount:.2f}. Please enter product code."
            return "Invalid coin amount."
        elif self.state in ['INSERTING_COIN', 'SELECTING_PRODUCT']:
            if amount > 0:
                self.inserted_amount += amount
                return f"${amount:.2f} inserted. Current amount: ${self.inserted_amount:.2f}."
            return "Invalid coin amount."
        return "Cannot insert coin at this time."

    def select_product(self, product_code):
        if self.state not in ['INSERTING_COIN', 'SELECTING_PRODUCT']:
            return "Insert coins first."
        if product_code not in self.stock:
            return "Invalid product code."
        product_info = self.stock[product_code]
        if product_info['quantity'] == 0:
            self.state = 'OUT_OF_STOCK'
            return f"{product_info['name']} is out of stock."
        if self.inserted_amount < product_info['price']:
            return f"Not enough money. ${product_info['price']:.2f} required for {product_info['name']}."
        self.selected_product_code = product_code
        self.state = 'DISPENSING'
        return f"{product_info['name']} selected. Price: ${product_info['price']:.2f}. Click 'Dispense'."

    def dispense(self):
        if self.state == 'DISPENSING' and self.selected_product_code:
            product_info = self.stock[self.selected_product_code]
            change = self.inserted_amount - product_info['price']
            self.stock[self.selected_product_code]['quantity'] -= 1
            product_name = product_info['name']
            self.selected_product_code = None
            self.inserted_amount = 0.0
            self.state = 'IDLE'
            if change > 0:
                return f"{product_name} dispensed. Change: ${change:.2f}. Thank you!"
            else:
                return f"{product_name} dispensed. Thank you!"
        return "No product to dispense."

    def refill(self, product_code, amount):
        if product_code in self.stock:
            self.stock[product_code]['quantity'] += amount
            if self.state == 'OUT_OF_STOCK' and self.stock[product_code]['quantity'] > 0:
                self.state = 'IDLE'
            return f"{self.stock[product_code]['name']} refilled by {amount}."
        return "Invalid product code."

    def cancel(self):
        if self.state in ['INSERTING_COIN', 'SELECTING_PRODUCT', 'DISPENSING']:
            refund = self.inserted_amount
            self.inserted_amount = 0.0
            self.selected_product_code = None
            self.state = 'IDLE'
            return f"Transaction cancelled. Refund: ${refund:.2f} returned."
        return "Nothing to cancel."

    def get_state(self):
        return self.state

    def get_stock(self):
        stock_info = {}
        for code, details in self.stock.items():
            stock_info[details['name']] = details['quantity']
        return stock_info.copy()

    def get_product_price(self, product_code):
        if product_code in self.stock:
            return self.stock[product_code]['price']
        return None

    def get_inserted_amount(self):
        return self.inserted_amount

    def get_product_name(self, product_code):
        if product_code in self.stock:
            return self.stock[product_code]['name']
        return None

class VendingMachineGUI:
    def __init__(self, master):
        self.vm = VendingMachine()
        self.master = master
        master.title("Vending Machine")
        master.geometry("450x700")  # Increased height

        self.label = tk.Label(master, text="Welcome!", font=("Helvetica", 14))
        self.label.pack(pady=10)

        self.output = tk.Text(master, height=6, width=40, wrap=tk.WORD, bg="#f0f0f0")
        self.output.pack(pady=5)

        self.inserted_amount_label = tk.Label(master, text="Inserted: $0.00", font=("Helvetica", 12))
        self.inserted_amount_label.pack(pady=5)

        tk.Label(master, text="Insert Coins:").pack()
        coin_frame = tk.Frame(master)
        coin_frame.pack()
        coins = [0.25, 0.50, 1.00]
        for coin in coins:
            btn = tk.Button(coin_frame, text=f"${coin:.2f}", width=8,
                            command=lambda c=coin: self.insert_coin(c), bg="gold")
            btn.pack(side=tk.LEFT, padx=5)

        tk.Label(master, text="Enter Product Code:").pack(pady=5)
        self.product_code_entry = tk.Entry(master, width=10)
        self.product_code_entry.pack()

        keypad_frame = tk.Frame(master)
        keypad_frame.pack(pady=5)
        buttons = [
            ('1', 1, 0), ('2', 1, 1), ('3', 1, 2),
            ('4', 2, 0), ('5', 2, 1), ('6', 2, 2),
            ('7', 3, 0), ('8', 3, 1), ('9', 3, 2),
            ('*', 4, 0), ('0', 4, 1), ('#', 4, 2)
        ]
        for (text, row, col) in buttons:
            btn = tk.Button(keypad_frame, text=text, width=5,
                            command=lambda t=text: self.enter_code(t))
            btn.grid(row=row, column=col, padx=2, pady=2)

        self.select_code_button = tk.Button(master, text="Select Product", command=self.select_product_by_code, width=20)
        self.select_code_button.pack(pady=5)

        self.dispense_button = tk.Button(master, text="Dispense", command=self.dispense, width=20, bg="lightgreen", state=tk.DISABLED)
        self.dispense_button.pack(pady=5)

        self.cancel_button = tk.Button(master, text="Cancel Transaction", command=self.cancel, width=20)
        self.cancel_button.pack(pady=5)

        self.stock_button = tk.Button(master, text="View Stock", command=self.show_stock, width=20)
        self.stock_button.pack(pady=5)

        refill_frame = tk.Frame(master)
        refill_frame.pack(pady=5)
        tk.Label(refill_frame, text="Refill Options (by Code):").pack()
        refill_buttons = [("A1", "Soda", 3), ("B2", "Chips", 3), ("C3", "Candy", 3)]
        for code, name, amount in refill_buttons:
            btn = tk.Button(refill_frame, text=f"Refill {name} ({code}, +{amount})",
                            command=lambda c=code, a=amount: self.refill(c, a), width=30)
            btn.pack(pady=2)

        self.state_label = tk.Label(master, text="Current State: IDLE", font=("Helvetica", 12))
        self.state_label.pack(pady=10)

        self.current_code = ""

    def update_output(self, message):
        self.output.delete(1.0, tk.END)
        self.output.insert(tk.END, message)
        self.state_label.config(text=f"Current State: {self.vm.get_state()}")
        self.inserted_amount_label.config(text=f"Inserted: ${self.vm.get_inserted_amount():.2f}")
        if self.vm.get_state() == 'DISPENSING':
            self.dispense_button.config(state=tk.NORMAL)
        else:
            self.dispense_button.config(state=tk.DISABLED)

    def insert_coin(self, amount):
        result = self.vm.insert_coin(amount)
        self.update_output(result)

    def enter_code(self, digit):
        self.current_code += digit
        self.product_code_entry.delete(0, tk.END)
        self.product_code_entry.insert(0, self.current_code)

    def select_product_by_code(self):
        code = self.product_code_entry.get()
        result = self.vm.select_product(code)
        self.update_output(result)
        self.current_code = ""
        self.product_code_entry.delete(0, tk.END)

    def dispense(self):
        result = self.vm.dispense()
        self.update_output(result)

    def cancel(self):
        result = self.vm.cancel()
        self.update_output(result)

    def show_stock(self):
        stock = self.vm.get_stock()
        message = "Current Stock:\n" + "\n".join(f"{k}: {v}" for k, v in stock.items())
        messagebox.showinfo("Stock Info", message)

    def refill(self, product_code, amount):
        result = self.vm.refill(product_code, amount)
        self.update_output(result)

if __name__ == '__main__':
    import sys
    if len(sys.argv) > 1 and sys.argv[1] == 'test':
        import unittest

        class TestVendingMachine(unittest.TestCase):
            def setUp(self):
                self.vm = VendingMachine()

            def test_initial_state(self):
                self.assertEqual(self.vm.get_state(), 'IDLE')

            def test_insert_coin(self):
                result = self.vm.insert_coin(0.50)
                self.assertEqual(self.vm.get_state(), 'INSERTING_COIN')
                self.assertEqual(self.vm.get_inserted_amount(), 0.50)
                self.assertIn("$0.50 inserted", result)

            def test_select_product_by_code_not_enough_money(self):
                self.vm.insert_coin(0.25)
                result = self.vm.select_product('A1')
                self.assertIn("Not enough money", result)
                self.assertEqual(self.vm.get_state(), 'SELECTING_PRODUCT')

            def test_select_product_by_code_out_of_stock(self):
                self.vm.insert_coin(1.00)
                result = self.vm.select_product('C3')
                self.assertIn("out of stock", result)
                self.assertEqual(self.vm.get_state(), 'OUT_OF_STOCK')

            def test_successful_dispense_by_code(self):
                self.vm.insert_coin(1.00)
                self.vm.select_product('A1')
                result = self.vm.dispense()
                self.assertEqual(self.vm.get_state(), 'IDLE')
                self.assertEqual(self.vm.stock['A1']['quantity'], 1)
                self.assertIn("Soda dispensed", result)
                self.assertEqual(self.vm.get_inserted_amount(), 0.0)

            def test_refill_by_code(self):
                self.vm.refill('C3', 2)
                self.assertEqual(self.vm.stock['C3']['quantity'], 2)

            def test_invalid_product_code(self):
                self.vm.insert_coin(1.00)
                result = self.vm.select_product('D4')
                self.assertIn("Invalid product code", result)

        unittest.main(argv=['first-arg-is-ignored'], exit=False)
    else:
        root = tk.Tk()
        app = VendingMachineGUI(root)
        root.mainloop()